In [3]:
import numpy as np
import pandas as pd
import pdb
import os
import math

In [25]:
os.getcwd()

'/csc/epitkane/projects/muat/preprocessing/notebook/gene_exp'

In [4]:
#requirements:
metadata = pd.read_csv('/csc/epitkane/projects/muat/extfile/metadata_icgc_pcawg.tsv',sep='\t',index_col=0) 
dictMutation = pd.read_csv('/csc/epitkane/projects/muat/extfile//dictMutation.csv',index_col=0)
dictChpos = pd.read_csv('/csc/epitkane/projects/muat/extfile/dictChpos.csv',index_col=0)
dictGES = pd.read_csv('/csc/epitkane/projects/muat/extfile/dictGES.csv',index_col=0)

In [5]:
sourcedir = '/csc/epitkane/projects/PCAWG20191001/data/modified_data/raw/indelmeisubsvperclass/'

In [6]:
all_class = os.listdir(sourcedir)

In [7]:
pcawg_dir = '/csc/epitkane/projects/PCAWG20191001/data/modified_data/train/allclasses/newformat/'

In [23]:
all_class[24]

'Liver-HCC'

In [24]:
for i in range(24,len(all_class)):
    all_samples = os.listdir(sourcedir + all_class[i])
    
    pcawg_histology = all_class[i]
    os.makedirs(pcawg_dir + pcawg_histology, exist_ok=True)
    for j in range(0,len(all_samples)):
        read_sample = pd.read_csv(sourcedir + all_class[i] + '/' + all_samples[j])
        
        somatic_files_persamples = read_sample
        
        pcawg_sample = all_samples[j][:-4]
        
        #1) simplifing mutation information
        simplified_mutation = []
        for k in range(0,len(somatic_files_persamples)):
            onemut = somatic_files_persamples.iloc[k]
            trp = onemut['seq'][int(len(onemut['seq'])/2)-1:int(len(onemut['seq'])/2)+2]
            ps = math.floor(onemut['pos'] / 1000000)
            chompos = str(onemut['chrom']) + '_' + str(ps)
            rt = onemut['rt']
            ges = str(onemut['genic']) + '_' + str(onemut['exonic']) + '_' + str(onemut['strand'])
            try:
                typ = dictMutation.loc[dictMutation['triplet']==trp]['mut_type'].values[0]
            except:
                typ = 'Neg'

            tuple_onerow = (trp,chompos,rt,ges,typ)
            simplified_mutation.append(tuple_onerow)
        pd_new = pd.DataFrame(simplified_mutation)
        pd_new.columns = ['seq','chrompos','rt','ges','mut_type'] 
        #save simplified mutation
        #new_files = 'new_' + pcawg_sample + '.csv'
        
        #2) tokenization from simplified mutation files
        mergetriplet = pd_new.merge(dictMutation, left_on='seq', right_on='triplet', how='left',suffixes=('', '_y'))
        mergetriplet.drop(mergetriplet.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        mergeges = mergetriplet.merge(dictGES, left_on='ges', right_on='ges', how='left',suffixes=('', '_y'))
        mergeges.drop(mergeges.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        mergeges = mergeges.rename(columns={"token": "gestoken"})
        mergechrompos = mergeges.merge(dictChpos, left_on='chrompos', right_on='chrompos', how='left',suffixes=('', '_y'))
        mergechrompos.drop(mergechrompos.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        mergechrompos = mergechrompos.rename(columns={"token": "postoken"})
        token_data = mergechrompos[['triplettoken','postoken','gestoken','rt','mut_type']]
        
        SNVonly = token_data.loc[token_data['mut_type']=='SNV']
        SNVonly = SNVonly.drop(columns=['mut_type'])
        MNVonly = token_data.loc[token_data['mut_type']=='MNV']
        MNVonly = MNVonly.drop(columns=['mut_type'])
        indelonly = token_data.loc[token_data['mut_type']=='indel']
        indelonly = indelonly.drop(columns=['mut_type'])
        MEISVonly = token_data.loc[token_data['mut_type'].isin(['MEI','SV'])]
        Negonly = token_data.loc[token_data['mut_type']=='Normal']
        Negonly = Negonly.drop(columns=['mut_type'])
        
        SNVonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'SNV_' + pcawg_sample + '.csv')
        MNVonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'MNV_' + pcawg_sample + '.csv')
        indelonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'indel_' + pcawg_sample + '.csv')
        MEISVonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'MEISV_' + pcawg_sample + '.csv')
        Negonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'Normal_' + pcawg_sample + '.csv')
        pd_count = pd.DataFrame([len(SNVonly),len(MNVonly),len(indelonly),len(MEISVonly),len(Negonly)])
        pd_count.to_csv(pcawg_dir + pcawg_histology + '/' + 'count_' + pcawg_sample + '.csv')
        
        print(pcawg_histology + '/' + pcawg_sample)        

Liver-HCC/f4025b40-a249-487f-8e93-7a140bcf57c1
Liver-HCC/7eb0ee38-fbe2-49ea-ba63-aa13825eca6e
Liver-HCC/8cf713ff-256b-4411-a1b4-5bf6f445b747
Liver-HCC/cb573c96-f6d4-4897-8919-9827f623b6a7
Liver-HCC/d3b72600-a888-4ef6-ac5d-661c105405dd
Liver-HCC/468083cb-95c3-47c1-9d96-79bcf7bf3f26
Liver-HCC/36e1d9cc-32ec-4a0a-8fb1-c46f058a6fb8
Liver-HCC/4c3eda94-56f4-46d3-8c88-50d476e773f6
Liver-HCC/b9098b7c-eab0-4ef5-b968-8d3024bf32f7
Liver-HCC/305a5074-5384-45a0-985d-289b178d09f6
Liver-HCC/5d58b984-e0b8-4237-a576-0e9d4079599e
Liver-HCC/f1a4f60a-5228-47cd-883f-ff8d2dfe1633
Liver-HCC/a2f143d6-ba07-4ff0-960d-b29c3c716665
Liver-HCC/47e5ff56-9534-4fcf-a593-33027e877f93
Liver-HCC/6b50391f-d890-4ca0-b942-beab0f8bf1c9
Liver-HCC/352fbbb4-88a5-4354-b1fa-3a01da3fbfa7
Liver-HCC/d252f328-4583-4e97-9a71-bb2885f06f73
Liver-HCC/c36213e4-c057-46f5-ada5-f1f921855bb4
Liver-HCC/1127b561-ea40-4d5e-95df-daa0a5ebc1e4
Liver-HCC/c016d025-3c92-41c5-b846-493b1fcce79e
Liver-HCC/c285c2fa-24b4-47a1-874d-86e74b002b05
Liver-HCC/484

/csc/epitkane/software/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Skin-Melanoma/76a0d9c9-5e69-44e8-9ed2-6d2e387803fc
Skin-Melanoma/16d5519e-ecb9-4fc8-81f1-e0e4adf722a8
Skin-Melanoma/5c3def3a-b515-41f6-8157-681b963534e7
Skin-Melanoma/08b5d0e4-4661-460e-a9f7-f2e687414711
Skin-Melanoma/58d3e932-b4a3-4cac-a6ac-73160e354a15
Skin-Melanoma/108749d2-5c62-4ef1-92df-aec6941ba53b
Skin-Melanoma/1d4a091d-fe65-49c0-8810-5a95243b108a
Skin-Melanoma/98e8f23c-5970-4fce-9551-4b11a772fe1b
Skin-Melanoma/a6e24c2a-26b9-409c-bbcd-a8fcdd1e437c
Skin-Melanoma/561fd34c-7c7d-4df0-bbfc-3d31147ca562
Skin-Melanoma/7ba70717-d65e-4a45-af79-60a976770526
Skin-Melanoma/c95a2b1b-726c-4608-9fff-d57b6f1aa75a
Skin-Melanoma/f24688a9-7952-49d8-ba80-2bbad472379c
Skin-Melanoma/5912d4b3-2d5d-4b98-b967-1b4ff177097a
Skin-Melanoma/6d4a548a-7e30-4b6b-9a13-72827eed439c
Skin-Melanoma/25103371-28ac-4f43-b0e2-2d376d0dcb84
Skin-Melanoma/28839c75-90a8-493f-b658-8c63e0ebd324
Skin-Melanoma/22d67778-61fc-4f15-95b8-7e7c6cc7112b
Skin-Melanoma/63762458-902a-4329-a823-703b54cb5f9d
Skin-Melanoma/51893d3f-e7f3-43f

In [15]:
pd_new

,seq,chrompos,rt,ges,mut_type
0,T%C,19_18,77.894100,1.0_0.0_+,SNV
1,A$C,1_70,35.886848,1.0_0.0_+,SNV
2,T%T,7_54,27.501750,0.0_0.0_=,SNV
3,T^A,20_17,41.822851,0.0_0.0_=,SNV
4,T%A,13_51,54.375898,0.0_0.0_=,SNV
...,...,...,...,...,...
32467,GDA,8_15,18.800451,0.0_0.0_=,SV
32468,CIT,8_16,24.003400,1.0_0.0_-,SV
32469,AIA,8_16,24.851251,1.0_0.0_+,SV
32470,GPC,8_31,21.398200,1.0_0.0_+,SV


In [16]:

#save token data
#token_files = 'token_' + pcawg_sample + '.csv'
#token_data.to_csv(pcawg_dir + pcawg_histology + '/' + token_files)

NameError: name 'pcawg_sample' is not defined

In [17]:
token_data

,triplettoken,postoken,gestoken,rt,mut_type
0,30,2560,12,77.894100,SNV
1,2,71,12,35.886848,SNV
2,32,1262,10,27.501750,SNV
3,45,2617,10,41.822851,SNV
4,29,2067,10,54.375898,SNV
...,...,...,...,...,...
32467,3462,1381,10,18.800451,SV
32468,3485,1382,13,24.003400,SV
32469,3470,1382,12,24.851251,SV
32470,3491,1397,12,21.398200,SV


In [18]:
SNVonly = token_data.loc[token_data['mut_type']=='SNV']
SNVonly = SNVonly.drop(columns=['mut_type'])
MNVonly = token_data.loc[token_data['mut_type']=='MNV']
MNVonly = MNVonly.drop(columns=['mut_type'])
indelonly = token_data.loc[token_data['mut_type']=='indel']
indelonly = indelonly.drop(columns=['mut_type'])
MEISVonly = token_data.loc[token_data['mut_type'].isin(['MEI','SV'])]
Negonly = token_data.loc[token_data['mut_type']=='Normal']
Negonly = Negonly.drop(columns=['mut_type'])